In [1]:
import tkinter as tk
from tkinter import Label, Button, filedialog, simpledialog, messagebox
from PIL import Image

def genData(data):
    newd = []
    for i in data:
        newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pixels = [value for value in next(imdata)[:3] +
                  next(imdata)[:3] +
                  next(imdata)[:3]]
        
        for j in range(8):
            if (datalist[i][j] == '0' and pixels[j] % 2 != 0):
                pixels[j] -= 1
            elif (datalist[i][j] == '1' and pixels[j] % 2 == 0):
                if pixels[j] != 0:
                    pixels[j] -= 1
                else:
                    pixels[j] += 1
        
        if i == lendata - 1:
            if pixels[-1] % 2 == 0:
                if pixels[-1] != 0:
                    pixels[-1] -= 1
                else:
                    pixels[-1] += 1
        else:
            if pixels[-1] % 2 != 0:
                pixels[-1] -= 1

        pixels = tuple(pixels)
        yield pixels[0:3]
        yield pixels[3:6]
        yield pixels[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)
    
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if x == w - 1:
            x = 0
            y += 1
        else:
            x += 1

def encode_image():
    img_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if not img_path:
        return

    image = Image.open(img_path, 'r')
    data = simpledialog.askstring("Input", "Enter data to be encoded:")
    if not data:
        messagebox.showerror("Error", "Data is empty")
        return

    newimg = image.copy()
    encode_enc(newimg, data)
    
    save_path = filedialog.asksaveasfilename(title="Save Encoded Image", defaultextension=".png", filetypes=[("PNG Files", "*.png"), ("JPEG Files", "*.jpg;*.jpeg")])
    if save_path:
        newimg.save(save_path)
        messagebox.showinfo("Success", "Image encoded and saved successfully")

def decode_image():
    img_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if not img_path:
        return

    image = Image.open(img_path, 'r')
    
    try:
        decoded_data = decode(image)
        messagebox.showinfo("Decoded Data", f"Decoded data: {decoded_data}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to decode image: {e}")

def decode(image):
    data = ''
    imgdata = iter(image.getdata())
    
    while True:
        try:
            pixels = [value for value in next(imgdata)[:3] +
                      next(imgdata)[:3] +
                      next(imgdata)[:3]]
        except StopIteration:
            break
        
        binstr = ''
        
        for i in pixels[:8]:
            if i % 2 == 0:
                binstr += '0'
            else:
                binstr += '1'
                
        data += chr(int(binstr, 2))
        if pixels[-1] % 2 != 0:
            return data

root = tk.Tk()
root.title('Project Image Steganography')
root.geometry('400x200')
root.resizable(0, 0)
root.config(bg='black')

Label(root, text='Project Image Steganography', font=('Comic Sans MS', 15), bg='white', wraplength=300).pack(pady=10)
Button(root, text='ENCODE', width=25, font=('Times New Roman', 13), bg='white', command=encode_image).pack(pady=10)
Button(root, text='DECODE', width=25, font=('Times New Roman', 13), bg='white', command=decode_image).pack(pady=10)

root.mainloop()
